# Named Entity Recognition (NER)

ဒီ notebook ဟာ [AI for Beginners Curriculum](http://aka.ms/ai-beginners) မှာပါဝင်တဲ့အပိုင်းတစ်ခုဖြစ်ပါတယ်။

ဒီဥပမာမှာတော့ [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) Dataset ကို အသုံးပြုပြီး NER မော်ဒယ်ကို ဘယ်လို training လုပ်မလဲဆိုတာကို လေ့လာသင်ယူမှာဖြစ်ပါတယ်။ စတင်လုပ်ဆောင်ရန်မတိုင်မီ [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) ဖိုင်ကို လက်ရှိ directory ထဲကို download လုပ်ထားပါ။


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## ဒေတာစုပုံကို ပြင်ဆင်ခြင်း

အရင်ဆုံး ဒေတာစုပုံကို dataframe အဖြစ် ဖတ်ရှုမည်ဖြစ်သည်။ Pandas ကို အသုံးပြုခြင်းအကြောင်း ပိုမိုလေ့လာလိုပါက [Data Science for Beginners](http://aka.ms/datascience-beginners) တွင်ရှိသော [ဒေတာလုပ်ငန်းစဉ်ဆိုင်ရာ သင်ခန်းစာ](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) ကို သွားရောက်ကြည့်ရှုနိုင်ပါသည်။


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

အခုတော့ ကျွန်တော်တို့ စကားလုံးများနဲ့ အတူတူလုပ်ရမယ်။ ရိုးရှင်းစွာအတွက် ကျွန်တော်တို့ စကားလုံးများကို frequency မထည့်သွင်းစဉ်းစားဘဲ vocabulary ဖန်တီးပါမယ်။ အမှန်တကယ်မှာတော့ သင် Keras vectorizer ကို အသုံးပြုပြီး စကားလုံးအရေအတွက်ကို ကန့်သတ်ချင်နိုင်ပါတယ်။


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

ကျွန်ုပ်တို့သည် လေ့ကျင့်ရန်အတွက် စာကြောင်းများ၏ ဒေတာစနစ်တစ်ခု ဖန်တီးရန် လိုအပ်ပါသည်။ မူရင်းဒေတာစနစ်ကို loop ဖြင့် ဖြတ်သန်းပြီး တစ်ခုချင်းစီသော စာကြောင်းများကို `X` (စကားလုံးများ၏ စာရင်းများ) နှင့် `Y` (token များ၏ စာရင်း) အဖြစ် ခွဲထုတ်ပါ။


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

အလွယ်တကူအောင်၊ ကျွန်ုပ်တို့သည် စာကြောင်းများအားလုံးကို အများဆုံးအရှည်သို့ 0 တိုကင်များဖြင့် ဖြည့်စွက်မည်။ အမှန်တကယ်ဘဝတွင်၊ ကျွန်ုပ်တို့သည် ပိုမိုလှပသောနည်းလမ်းတစ်ခုကို အသုံးပြုလိုနိုင်ပြီး၊ sequence များကို တစ်ခုချင်းစီ minibatch အတွင်းတွင်သာ ဖြည့်စွက်မည်။


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Token Classification Network ကို သတ်မှတ်ခြင်း

Token classification အတွက် two-layer bidirectional LSTM network ကို အသုံးပြုမည်ဖြစ်သည်။ နောက်ဆုံး LSTM layer ၏ output တစ်ခုချင်းစီကို dense classifier ဖြင့် လျှောက်ထားရန်အတွက် `TimeDistributed` construction ကို အသုံးပြုမည်ဖြစ်ပြီး၊ ၎င်းသည် LSTM ၏ output တစ်ခုချင်းစီတွင် တူညီသော dense layer ကို အဆင့်တိုင်းတွင် ထပ်တူထပ်မျှ လျှောက်ထားပေးသည်။


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

ကျွန်ုပ်တို့၏ dataset အတွက် `maxlen` ကို ထည့်သွင်းသတ်မှတ်ထားသည်ကို ဤနေရာတွင် သတိပြုပါ - network သည် အလျားအမျိုးမျိုးရှိ sequence များကို ကိုင်တွယ်နိုင်စေရန်လိုပါက၊ network ကို သတ်မှတ်ရာတွင် ပိုမိုထူးခြားသော နည်းလမ်းများကို အသုံးပြုရန် လိုအပ်ပါသည်။

အခုတော့ model ကို training ပြုလုပ်ကြမည်။ အချိန်တိုအတွင်း အဆင်ပြေစေရန်၊ ကျွန်ုပ်တို့သည် epoch တစ်ခုသာ training ပြုလုပ်မည်ဖြစ်သော်လည်း၊ သင်သည် ပိုမိုကြာရှည်စွာ training ပြုလုပ်ရန် ကြိုးစားနိုင်ပါသည်။ ထို့အပြင်၊ dataset ၏ အချို့ကို training dataset အဖြစ် ခွဲထုတ်ထားပြီး၊ validation accuracy ကို ကြည့်ရှုနိုင်ရန် လုပ်ဆောင်နိုင်ပါသည်။


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## ရလဒ်ကို စမ်းသပ်ခြင်း

အခုတော့ ကျွန်တော်တို့ရဲ့ entity recognition မော်ဒယ်ကို နမူနာ စာကြောင်းတစ်ကြောင်းမှာ အလုပ်လုပ်ပုံကို ကြည့်လိုက်ရအောင်:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## အဓိကအချက်

ရိုးရှင်းတဲ့ LSTM မော်ဒယ်တောင် NER မှာ သင့်တော်တဲ့ရလဒ်တွေကို ပြသနိုင်ပါတယ်။ သို့သော် ပိုမိုကောင်းမွန်တဲ့ရလဒ်တွေကို ရရှိစေဖို့အတွက် BERT ကဲ့သို့သော ကြီးမားတဲ့ pre-trained language models ကို အသုံးပြုလိုက်ပါ။ Huggingface Transformers library ကို အသုံးပြုပြီး NER အတွက် BERT ကို လေ့ကျင့်ခြင်းကို [ဒီမှာ](https://huggingface.co/course/chapter7/2?fw=pt) ဖော်ပြထားပါတယ်။



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူလဘာသာစကားဖြင့် အာဏာတရ အရင်းအမြစ်အဖြစ် ရှုလေ့လာသင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ပရော်ဖက်ရှင်နယ် ဘာသာပြန်မှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွတ်များ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
